# Potential SNIa Menagerie

Author: Melissa Graham

Create an image for the paper that shows lightcurves of potential Type Ia supernovae.

In [ ]:
# import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import pandas as pd

# from copy import deepcopy
import time

## Load the polygons data

Type Ia SNe occupy a certain region in the parameter space of time span _vs._ amplitude.

MLG derived polygons in this region of parameter space separately, and we read them in now.

In [ ]:
fnm = 'MLG_polygons_tspan_lcamp.dat'
poly_tspans = np.loadtxt( fnm, dtype='float', usecols=(0) )
poly_lcamps = np.loadtxt( fnm, dtype='float', usecols=(1,2,3,4,5,6), unpack=True)

## Load the lightcurve parameters data

In [ ]:
fnm = 'candidate_nightly_epochs_files/candidate_lightcurve_parameters.dat'

paramLC_field = np.loadtxt( fnm, dtype='str', usecols=(0) )
paramLC_candid = np.loadtxt( fnm, dtype='str', usecols=(1) )
paramLC_tspan = np.loadtxt( fnm, dtype='float', usecols=(2) )
paramLC_tspan_g = np.loadtxt( fnm, dtype='float', usecols=(3) )
paramLC_tspan_r = np.loadtxt( fnm, dtype='float', usecols=(4) )
paramLC_tspan_i = np.loadtxt( fnm, dtype='float', usecols=(5) )
paramLC_minmag = np.loadtxt( fnm, dtype='float', usecols=(6) )
paramLC_minmag_g = np.loadtxt( fnm, dtype='float', usecols=(7) )
paramLC_minmag_r = np.loadtxt( fnm, dtype='float', usecols=(8) )
paramLC_minmag_i = np.loadtxt( fnm, dtype='float', usecols=(9) )
paramLC_amp = np.loadtxt( fnm, dtype='float', usecols=(10) )
paramLC_amp_g = np.loadtxt( fnm, dtype='float', usecols=(11) )
paramLC_amp_r = np.loadtxt( fnm, dtype='float', usecols=(12) )
paramLC_amp_i = np.loadtxt( fnm, dtype='float', usecols=(13) )
paramLC_neps = np.loadtxt( fnm, dtype='int', usecols=(14) )
paramLC_neps_g = np.loadtxt( fnm, dtype='int', usecols=(15) )
paramLC_neps_r = np.loadtxt( fnm, dtype='int', usecols=(16) )
paramLC_neps_i = np.loadtxt( fnm, dtype='int', usecols=(17) )

del fnm

In [ ]:
fig, ax = plt.subplots( 1, 3, figsize=(16,3), sharey=True )

tx = np.where( (cand_field == 'COSMOS') & (cand_tspanNE > 10) & (cand_lcampNE > 0.5) & (cand_numNE > 5) )[0]
print(len(tx))
ax[0].plot( cand_tspanNE_g[tx], cand_lcampNE_g[tx], 'o', ms=3, mew=0, alpha=0.5, color=clr[0] )
ax[1].plot( cand_tspanNE_r[tx], cand_lcampNE_r[tx], 'o', ms=3, mew=0, alpha=0.5, color=clr[1] )
ax[2].plot( cand_tspanNE_i[tx], cand_lcampNE_i[tx], 'o', ms=3, mew=0, alpha=0.5, color=clr[2] )
del tx

tx = np.where( (cand_field == 'ELAIS') & (cand_tspanNE > 10) & (cand_lcampNE > 0.5) & (cand_numNE > 5) )[0]
print(len(tx))
ax[0].plot( cand_tspanNE_g[tx], cand_lcampNE_g[tx], 'o', ms=3, mew=0, alpha=0.5, color=clr[3] )
ax[1].plot( cand_tspanNE_r[tx], cand_lcampNE_r[tx], 'o', ms=3, mew=0, alpha=0.5, color=clr[4] )
ax[2].plot( cand_tspanNE_i[tx], cand_lcampNE_i[tx], 'o', ms=3, mew=0, alpha=0.5, color=clr[5] )
del tx

### plot the polygon ranges from template light curves
for a in range(3):
    i = a * 2
    j = (a * 2) + 1
    x = poly_tspans[:]
    y1 = poly_lcamps[i,:]
    y2 = poly_lcamps[j,:]
    ax[a].plot( x, y1, lw=1, color='black')
    ax[a].plot( x, y2, lw=1, color='black')

### create an even easier cut and apply it
x = poly_tspans[:]
slopes = np.asarray([1.5/40, 1.5/40, 1.5/40],dtype='float')
yints  = np.asarray([1.5-slopes[0]*70, 1.5-slopes[1]*70, 1.5-slopes[2]*80],dtype='float')
for f in range(3):
    y = slopes[f]*x + yints[f]
    ax[f].plot( x, y, lw=1, ls='dashed')
del x, y

cand_potSNIa = np.zeros( len(cand_field), dtype='int' )
tx = np.where( (cand_tspanNE > 10) & (cand_lcampNE > 0.5) & (cand_numNE > 5) )[0]
for t in tx:
    cntr = 0
    glow = slopes[0]*cand_tspanNE_g[t] + yints[0]
    rlow = slopes[1]*cand_tspanNE_r[t] + yints[1]
    ilow = slopes[2]*cand_tspanNE_i[t] + yints[2]
    if (glow < cand_lcampNE_g[t]) & (cand_lcampNE_g[t]>0.5) & (cand_tspanNE_g[t]>10):
        cntr += 1
    if (rlow < cand_lcampNE_r[t]) & (cand_lcampNE_r[t]>0.5) & (cand_tspanNE_r[t]>10):
        cntr += 1
    if (ilow < cand_lcampNE_i[t]) & (cand_lcampNE_i[t]>0.5) & (cand_tspanNE_i[t]>10):
        cntr += 1
    if cntr >= 2:
        cand_potSNIa[t] = 1
    del glow,rlow,ilow,cntr
del tx, slopes, yints

### overplot the potential SNeIa
tx = np.where( (cand_potSNIa == 1) & (cand_field == 'COSMOS') )[0]
print('potential sneia in cosmos: ',len(tx))
ax[0].plot( cand_tspanNE_g[tx], cand_lcampNE_g[tx], '*', ms=8, mew=0, alpha=0.7, color=clr[0] )
ax[1].plot( cand_tspanNE_r[tx], cand_lcampNE_r[tx], '*', ms=8, mew=0, alpha=0.7, color=clr[1] )
ax[2].plot( cand_tspanNE_i[tx], cand_lcampNE_i[tx], '*', ms=8, mew=0, alpha=0.7, color=clr[2] )
del tx

tx = np.where( (cand_potSNIa == 1) & (cand_field == 'ELAIS') )[0]
print('potential sneia in elais: ',len(tx))
ax[0].plot( cand_tspanNE_g[tx], cand_lcampNE_g[tx], '*', ms=8, mew=0, alpha=0.7, color=clr[3] )
ax[1].plot( cand_tspanNE_r[tx], cand_lcampNE_r[tx], '*', ms=8, mew=0, alpha=0.7, color=clr[4] )
ax[2].plot( cand_tspanNE_i[tx], cand_lcampNE_i[tx], '*', ms=8, mew=0, alpha=0.7, color=clr[5] )
del tx

ax[0].set_xlim([0,150])
ax[1].set_xlim([0,150])
ax[2].set_xlim([0,150])
ax[0].set_ylim([0.,2.5])
ax[1].set_ylim([0.,2.5])
ax[2].set_ylim([0.,2.5])

ax[0].set_ylabel('amplitude')
ax[0].set_xlabel('time span')
ax[1].set_xlabel('time span')
ax[2].set_xlabel('time span')

plt.show()

In [ ]:
fig, ax = plt.subplots( 4,5, figsize=(16,8), sharey=True )
tx = np.where( (cand_potSNIa == 1) & (cand_field == 'COSMOS') )[0]
i = 0
j = 0
for t in tx:
    cx = np.where( clcNE_candid == cand_id[t] )[0]
    for f,filt in enumerate(['g','r','i']):
        fx = np.where( clcNE_fil[cx] == filt )[0]
        x = clcNE_mjd[cx[fx]]
        y = clcNE_mag[cx[fx]]
        yerr = clcNE_mage[cx[fx]]
        ax[i,j].errorbar(x, y, yerr=yerr, fmt='o', ms=3, mew=0, alpha=0.5, color=clr[f] )
        del x, y, yerr, fx
    ax[i,j].text(0.20, 0.9, cand_id[t], \
                 horizontalalignment='center', verticalalignment='center', transform=ax[i,j].transAxes)
    plt.gca().invert_yaxis()
    del cx
    j += 1
    if j > 4:
        j = 0
        i += 1

del tx,i,j

In [ ]:
fig, ax = plt.subplots( 4,5, figsize=(16,8), sharey=True )
tx = np.where( (cand_potSNIa == 1) & (cand_field == 'ELAIS') )[0]
i = 0
j = 0
for t in tx:
    cx = np.where( elcNE_candid == cand_id[t] )[0]
    for f,filt in enumerate(['g','r','i']):
        fx = np.where( elcNE_fil[cx] == filt )[0]
        x = elcNE_mjd[cx[fx]]
        y = elcNE_mag[cx[fx]]
        yerr = elcNE_mage[cx[fx]]
        ax[i,j].errorbar(x, y, yerr=yerr, fmt='o', ms=3, mew=0, alpha=0.5, color=clr[f] )
        del x, y, yerr, fx
    ax[i,j].text(0.20, 0.2, cand_id[t], \
                 horizontalalignment='center', verticalalignment='center', transform=ax[i,j].transAxes)
    plt.gca().invert_yaxis()
    del cx
    j += 1
    if j > 4:
        j = 0
        i += 1
del tx,i,j

In [ ]:
### menagerie
snia_menagerie = ['DC21jof','DC21dhgr','DC21dtwb','DC21dna','DC21gayy','DC21kgy',
                  'DC21kog','DC21dqik','DC21fber','DC21bkrj','DC21cove','DC21dslz','DC21mjrj',
                  'DC21cgfrp','DC21cgfhs','DC21cozcn','DC21cpcjp','DC21cwcwp','DC21cwtvj',
                  'DC21cwtlp','DC21cwxin','DC21cyddn','DC21drcko','DC21drcmp','DC21dsocp']
print(len(snia_menagerie))

In [ ]:
fig, ax = plt.subplots( 5,5, figsize=(12,12), sharey=True )
i = 0
j = 0
for s,sname in enumerate(snia_menagerie):
    c = np.where( cand_id == sname )[0]
    if cand_field[c] == 'ELAIS':
        cx = np.where( elcNE_candid == cand_id[c] )[0]
        for f,filt in enumerate(['g','r','i']):
            fx = np.where( elcNE_fil[cx] == filt )[0]
            x = elcNE_mjd[cx[fx]] - 59000
            y = elcNE_mag[cx[fx]] - (float(f)-1.0)*0.5
            yerr = elcNE_mage[cx[fx]]
            ax[i,j].errorbar(x, y, yerr=yerr, fmt='o', ms=3, mew=0, alpha=0.5, color=clr[f] )
            # del x, y, yerr, fx
        ax[i,j].plot(x[0], 99, 'o', color='white', label=sname)
        ax[i,j].legend(loc='best',frameon=False)
        # plt.gca().invert_yaxis()
        del cx
        j += 1
        if j > 4:
            j = 0
            i += 1
    elif cand_field[c] == 'COSMOS':
        cx = np.where( clcNE_candid == cand_id[c] )[0]
        for f,filt in enumerate(['g','r','i']):
            fx = np.where( clcNE_fil[cx] == filt )[0]
            x = clcNE_mjd[cx[fx]] - 59000
            y = clcNE_mag[cx[fx]] - (float(f)-1.0)*0.5
            yerr = clcNE_mage[cx[fx]]
            ax[i,j].errorbar(x, y, yerr=yerr, fmt='o', ms=3, mew=0, alpha=0.5, color=clr[f] )
            # del x, y, yerr, fx
        ax[i,j].plot(x[0], 99, 'o', color='white', label=sname)
        ax[i,j].legend(loc='best',frameon=False)
        # plt.gca().invert_yaxis()
        del cx
        j += 1
        if j > 4:
            j = 0
            i += 1
    del c
ax[0,0].text(0.2, 0.1, 'g+0.5', color=clr[0], \
             horizontalalignment='center', verticalalignment='center', transform=ax[0,0].transAxes)
ax[0,0].text(0.4, 0.1, 'r', color=clr[1], \
             horizontalalignment='center', verticalalignment='center', transform=ax[0,0].transAxes)
ax[0,0].text(0.6, 0.1, 'i-0.5', color=clr[2], \
             horizontalalignment='center', verticalalignment='center', transform=ax[0,0].transAxes)

ax[0,0].set_ylim([24.5,19.5])
ax[0,0].set_ylabel('magnitude')
ax[1,0].set_ylabel('magnitude')
ax[2,0].set_ylabel('magnitude')
ax[3,0].set_ylabel('magnitude')
ax[4,0].set_ylabel('magnitude')
ax[4,0].set_xlabel('MJD-59000')
ax[4,1].set_xlabel('MJD-59000')
ax[4,2].set_xlabel('MJD-59000')
ax[4,3].set_xlabel('MJD-59000')
ax[4,4].set_xlabel('MJD-59000')
del i,j

plt.savefig('candidate_summary_figures/snia_menagerie')